# Assignment 1 Part 1 - Genetic Algorithm

**Note**: The following assumes that the distance between two cities is the shortest path between their xy-coordinates.

## (1) Base algorithm

### 1. Preparation

* defining 'typealiases' for clearer code
* defining filename variable

In [2]:
from typing import List, Tuple


CityCoordinate = Tuple[float, float]
Route = List[int]

TSP_FILENAME = "d200-41.tsp"

### 2. Base genetic algorithm

I decided to go with an abstract genetic algorithm for TSPs, as to reduce code dupication when using different parameters. The ABC gives me the option to require methods from subclasses.

The parameters and required methods are explained in the docstring.

Additionally, the TSPGA imports the coordinates at `TSP_FILENAME` and calculates the fitness scores. The fitness score is calculated by using the distance. Invalid routes are penalized infinitely.

In [3]:
from abc import ABC, abstractmethod
from math import sqrt
from random import random
from time import time

class TSPGA(ABC):
    """
    The abstract base class of a genetic algorithm (GA) for solving Travelling Salesman Problem's (TSP).

    Parameters
        `population_size (int)`: An integer representing the population size.\n
        `parent_count (int)`: An integer representing the number of parents.\n
        `mutation_rate (float)`: A float representing the rate of mutation (0 to 1).\n

    Required methods
        `generate() -> List[Route]`: Generates an initial population.\n
        `select(List[Route], count: int) -> List[Route]`: Selects `count` individuals from provided candidates.\n
        `procreate(List[Route]) -> Route`: Creates a single child from the provided parents.\n
        `mutate (Route) -> Route`: Mutates a single route.
    """

    def __init__(self, 
                 population_size: int, 
                 parent_count: int, 
                 mutation_rate: float,
                 city_count: int):
        self.population_size = population_size
        self.parent_count = parent_count
        self.mutation_rate = mutation_rate
        self.city_count = city_count

        self.city_coordinates = self.import_city_coordinates()

    @abstractmethod
    def generate(self) -> List[Route]:
        pass

    @abstractmethod
    def select(self, candidates: List[Route], count: int) -> List[Route]:
        pass

    @abstractmethod
    def procreate(self, parents: List[Route]) -> Route:
        pass

    @abstractmethod
    def mutate(self, route: Route) -> Route:
        pass
    
    @property
    def execution_time(self) -> float:
        return self.end_time - self.start_time
    
    @property
    def result_fitness(self) -> float:
        return self.calculate_fitness(route=self.result)

    def execute(self, max_generations: int):
        """
        The method of the TSPGA to execute the algorithm.

        Parameter `max_generations (int)`: The maximum number of generations before execution finishes.

        Results are saved to the following properties:
            `result (Route)`: The resulting fittest route.
            `result_fitness (float)`: The fitness of `result`.
            `execution_time (float)`: The total time of execution (in seconds).
        """
        self.start_time = time()

        population: List[Route] = self.generate()

        for i in range(max_generations):
            self.result = min(population, key=self.calculate_fitness)

            parents = self.select(candidates=population, count=self.parent_count)
            
            # offsets the parent array to generate a new child for each parent
            offspring = [self.procreate(parents=parents[i:] + parents[:i]) for i in range(len(parents))]
            survivors = self.select(candidates=population + offspring, count=self.population_size)
            
            # mutates with a change of mutation_rate
            with_mutants = [self.mutate(route=route) if random() < self.mutation_rate else route for route in survivors]

            population = with_mutants.copy()

        self.end_time = time()

    def import_city_coordinates(self) -> List[CityCoordinate]:
        with open(TSP_FILENAME, 'r') as file:
            city_coordinates: List[CityCoordinate] = []

            for line in file:
                raw = line.strip().split("  ")
                x, y = float(raw[0]), float(raw[1])
                city_coordinates.append((x, y))

            return city_coordinates.copy()[:self.city_count]

    def calculate_fitness(self, route: Route) -> float:
        if not self.contains_all_cities(route=route): 
            return float('inf')
        
        # the distance of the last leg is added here
        distance: float = self.calculate_city_distance(first=self.city_coordinates[route[-1]], second=self.city_coordinates[route[0]])

        for i in range(self.city_count - 1):

            city_distance = self.calculate_city_distance(
                first=self.city_coordinates[route[i]], second=self.city_coordinates[route[i + 1]]
            )
            distance += city_distance
        
        return distance
    
    def calculate_city_distance(self, first: CityCoordinate, second: CityCoordinate) -> float:
        distance = sqrt((second[0] - first[0])**2 + (second[1] - first[1])**2)
        return distance
    
    def contains_all_cities(self, route: Route) -> bool:
        all_cities = set(range(self.city_count))
        return set(route) == all_cities

## (2) Initial Design

For the baseline algorithm I decided to use the following parameters:
* **Generation**: Random initial population (of valid routes).
* **Selection**: Parents are selected by highest fitness scores.
* **Procreation**: Procreation is implemented via equal cut-and-crossfill (to generate valid routes).
* **Mutation**: Mutation is carried out via two-element swapping.
* **Others**: population size 10, parent count 2, mutation rate of 5%.

**Note**: The experiments only use the first 50 cities for quicker execution.

### 1. Initial parameters

#### Generation

In generating initial routes, I decided to go with random routes. This was accomplished by creating the default route (first to last city in order) and applying random's shuffle function.

**Possible experiments**:
* Provide a fixed set of initial routes (like forwards, backwards).
* Incorporate coordinate data to generate better initial routes.

In [4]:
from random import shuffle


def generate_initial_population_randomly(population_size: int, city_count: int) -> List[Route]:
     def generate_random_route() -> Route:
          route: Route = list(range(city_count))
          shuffle(route)
          return route

     initial_routes: list[Route] = [generate_random_route() for _ in range(population_size)]

     return initial_routes

#### Selection

I decided to go with the same mechanism for selecting parents and survivors. Using different mechanisms seems a unnecessarily complicate things.

Starting off, I chose to select the fittest individuals.

**Possible experiments**:
* Random selection.
* Random selection with weighted fitness scores.

In [5]:
from typing import Callable


def select_by_fitness(candidates: List[Route], count: int, calculate_fitness: Callable[[Route], float]) -> List[Route]:
    candidates_in_ascending_order: List[Route] = sorted(candidates, key=calculate_fitness)
    parents: List[Route] = candidates_in_ascending_order[:count]
    return parents

#### Procreation

For now, I opted to go with equal cut-and-crossfill for procreation (all parents pass on equal amount of genes).

**Possible Experiments**:
* Random crossover points.
* Fitness-influenced crossover points.

In [6]:
def procreate_by_equal_cut_and_crossfill(parents: List[Route]) -> Route:
    parent_count = len(parents)
    city_count = len(parents[0])
    step = (city_count-1) // (parent_count-1)
    crossover_points = list(range(step, city_count, step)) 
    
    child = parents[0][:crossover_points[0]]
    for i in range(len(crossover_points)):
        start = crossover_points[i - 1] if i > 0 else 0
        end = crossover_points[i] if i < len(crossover_points) - 1 else len(crossover_points)

        for j in range(start, end):
            if parents[i][j] not in child[:end]:
                child.append(parents[i][j])

    # this adds the missing values to the child
    if len(child) < city_count: 
        child += list(set(range(city_count)) - set(child))

    return child

#### Mutation

For mutation I chose to go with a swap mutation. Implementation uses the random module.

**Possible experiments**:
* Using increment mutation (will generate illegal solutions).
* Swapping sub lists of random length.

In [7]:
from random import sample


def mutate_by_swapping(route: Route) -> Route:
    new_route = route.copy()
    pos1, pos2 = sample(range(len(new_route)), 2)
    new_route[pos1], new_route[pos2] = new_route[pos2], new_route[pos1]
    return new_route

### 2. TSP genetic algorithm v1

In [8]:
class V1(TSPGA):
    def __init__(self, 
                 population_size: int = 10, 
                 parent_count: int = 2, 
                 mutation_rate: float = 0.05,
                 city_count: int = 50):
        super().__init__(population_size, parent_count, mutation_rate, city_count)

    def generate(self) -> List[Route]:
        return generate_initial_population_randomly(
            population_size=self.population_size, 
            city_count=self.city_count
        )
    
    def select(self, candidates: List[Route], count: int) -> List[Route]:
        return select_by_fitness(
            candidates=candidates, 
            count=count, 
            calculate_fitness=self.calculate_fitness
        )
    
    def procreate(self, parents: List[Route]) -> Route:
        return procreate_by_equal_cut_and_crossfill(parents=parents)
    
    def mutate(self, route: Route) -> Route:
        return mutate_by_swapping(route=route)

### 3. Automatic measurements

Before experimenting, I needed to decide on some measurements. In the end I picked
1. the actual time of evolving through 100 generations.
2. the fitness of the best result after 100 generations.

With these two, I will run each algorithm 10 times. This should give a good estimate of parameter quality.

**Note**: These measurements are measured in the abstract genetic algorithm defined above.

In [9]:
def measure(title: str, tspga: TSPGA, max_generations: int = 100, run_count: int = 10):
    result_fitness_values, execution_times = [], []

    print()

    for i in range(run_count):
        tspga.execute(max_generations=max_generations)
        print(f"Execution {i+1} | Time: {tspga.execution_time:.3f}s | Fitness: {tspga.result_fitness:.3f}.", end="\r")
        result_fitness_values.append(tspga.result_fitness)
        execution_times.append(tspga.execution_time)

    print(" " * len(f"Execution {run_count} | Time: {tspga.execution_time:.3f}s | Fitness: {tspga.result_fitness:.3f}."), end="\r")

    average_result_fitness = sum(result_fitness_values) / run_count
    average_execution_time = sum(execution_times) / run_count
    
    print(title + ":")
    print(f" Average result fitness: {average_result_fitness:.3f}.")
    print(f" Average execution time: {average_execution_time:.3f}s.")

### 4. Generating a baseline

In [10]:
def measure_baseline():
    measure("BASELINE", tspga=V1())

measure_baseline()


BASELINE:                                     
 Average result fitness: 25.497.
 Average execution time: 0.036s.


## (3) Experimenting

### Changing the population size

In [11]:
measure_baseline()
measure("POPULATION SIZE 10", tspga=V1(population_size=25))
measure("POPULATION SIZE 50", tspga=V1(population_size=50))
measure("POPULATION SIZE 100", tspga=V1(population_size=100))


BASELINE:                                     
 Average result fitness: 26.724.
 Average execution time: 0.036s.

POPULATION SIZE 10:                           
 Average result fitness: 23.728.
 Average execution time: 0.085s.

POPULATION SIZE 50:                           
 Average result fitness: 23.069.
 Average execution time: 0.167s.

POPULATION SIZE 100:                          
 Average result fitness: 22.984.
 Average execution time: 0.332s.


**Execution time**: Execution time increases in a linear fashion with population size.  
**Result fitness**: Population size seems to improve resulting fitness, but only up to a point.

**Conclusion**: Taking into account the performance decrease brought on by higher population size, I decided on a population size of 25. This seems to bring a good increase in fitness score, whilst still executing reasonably quickly.

In [12]:
POPULATION_SIZE : int= 25

### Changing the parent count

In [13]:
measure_baseline()
measure("PARENT COUNT 5", tspga=V1(parent_count=5))
measure("PARENT COUNT 15", tspga=V1(parent_count=15))
measure("PARENT COUNT 25", tspga=V1(parent_count=25))


BASELINE:                                     
 Average result fitness: 25.644.
 Average execution time: 0.036s.

PARENT COUNT 5:                               
 Average result fitness: 25.010.
 Average execution time: 0.043s.

PARENT COUNT 15:                              
 Average result fitness: 21.974.
 Average execution time: 0.055s.

PARENT COUNT 25:                              
 Average result fitness: 22.255.
 Average execution time: 0.054s.


**Execution time**: Execution time seems to increase only marginally.  
**Result fitness**: The result fitness increases, but again only to a point.  

**Conclusion**: A parent count of around 15 seems to result in the best fitness values, whilst only marginally decreasing performance.

In [14]:
PARENT_COUNT = 15

### Changing mutation rate

In [15]:
measure_baseline()
measure("MUTATION RATE 0%", tspga=V1(mutation_rate=0))
measure("MUTATION RATE 25%", tspga=V1(mutation_rate=0.25))
measure("MUTATION RATE 100%", tspga=V1(mutation_rate=1))


BASELINE:                                     
 Average result fitness: 25.704.
 Average execution time: 0.036s.

MUTATION RATE 0%:                             
 Average result fitness: 30.873.
 Average execution time: 0.035s.

MUTATION RATE 25%:                            
 Average result fitness: 21.068.
 Average execution time: 0.036s.

MUTATION RATE 100%:                           
 Average result fitness: 24.330.
 Average execution time: 0.037s.


**Execution time**: Execution time seems to increase only marginally.  
**Result fitness**: Fitness increases at first, but then decreases again with a higher mutation rate.

**Conclusion**: The experiments point towards a higher mutation rate of around 25%.

In [16]:
MUTATION_RATE = 0.25

### IGNORE: Changing selection mechanism

For this experiment, I will use the random selection and random selection with fitness weights.

In [17]:
from random import choices

def select_randomly(candidates: List[Route], count: int) -> List[Route]:
    survivors: List[Route] = choices(candidates, k=count)
    return survivors

class V1_RandomSelection(V1):
    def select(self, candidates: List[Route], count: int) -> List[Route]:
        return select_randomly(candidates=candidates, count=count)

In [18]:
from random import choices

def select_randomly_with_fitness_weights(
        candidates: List[Route], count: int, calculate_fitness: Callable[[Route], float]
) -> List[Route]:
    survivors: List[Route] = choices(
        candidates,
        weights=[1 / calculate_fitness(candidate) + 1 for candidate in candidates],
        k=count
    )
    return survivors

class V1_FitnessWeightedRandomSelection(V1):
    def select(self, candidates: List[Route], count: int) -> List[Route]:
        return select_randomly_with_fitness_weights(
            candidates=candidates,
            count=count,
            calculate_fitness=self.calculate_fitness
        )

In [19]:
measure_baseline()
measure("RANDOM", tspga=V1_RandomSelection())
measure("RANDOM WITH FITNESS WEIGHTS", tspga=V1_FitnessWeightedRandomSelection())


BASELINE:                                     
 Average result fitness: 25.855.
 Average execution time: 0.036s.

RANDOM:                                       
 Average result fitness: 33.721.
 Average execution time: 0.012s.

RANDOM WITH FITNESS WEIGHTS:                  
 Average result fitness: 33.170.
 Average execution time: 0.036s.


**Execution time**: Execution time seems to increase noticeably for random selection, marginally when using fitness weights.  
**Result fitness**: Resulting fitness is a lot worse than in the baseline.

**Conclusion**: It seems, both approaches severely worsen resulting fitness. I will stay with the baseline mechanism.

### IGNORE: Changing procreation mechanism

In procreation i chose to do two experiments:
1. Using simple crossover (will generate illegal solutions).
2. Using random crossover points instead of equally crossing over all parents.

**Note**: Number 2. is generated using random's randint and removing duplicates -> Not all parents are crossed over into all children.

In [20]:
from random import randint


def procreate_by_simple_crossover(parents: List[Route]) -> Route:
    parent_count = len(parents)
    city_count = len(parents[0])

    step = (city_count-1) // (parent_count-1)
    crossover_points = list(range(step, city_count, step)) 
    
    child = parents[0].copy()
    for i in range(len(crossover_points)):
        start = crossover_points[i - 1] if i > 0 else 0
        end = crossover_points[i] if i < len(crossover_points) - 1 else len(crossover_points)

        child[start:end] = parents[i][start:end]

    return child

class V1_ProcreateBySimpleCrossover(V1):
    def procreate(self, parents: List[Route]) -> Route:
        return procreate_by_simple_crossover(parents=parents)

In [21]:
from random import randint


def procreate_by_random_cut_and_crossfill(parents: List[Route]) -> Route:
    parent_count = len(parents)
    city_count = len(parents[0])

    crossover_points = sorted(list(set(randint(0, city_count) for _ in range(parent_count))))
    
    child = parents[0][:crossover_points[0]]
    for i in range(len(crossover_points)):
        start = crossover_points[i - 1] if i > 0 else 0
        end = crossover_points[i] if i < len(crossover_points) - 1 else len(crossover_points)

        for j in range(start, end):
            if parents[i][j] not in child[:end]:
                child.append(parents[i][j])

    # this adds the missing values to the child
    if len(child) < city_count: 
        child += list(set(range(city_count)) - set(child))

    return child

class V1_ProcreateByCrossoverWithRandomPoints(V1):
    def procreate(self, parents: List[Route]) -> Route:
        return procreate_by_random_cut_and_crossfill(parents=parents)

In [22]:
measure_baseline()
measure("PROCREATE BY SIMPLE CROSSOVER", tspga=V1_ProcreateBySimpleCrossover())
measure("PROCREATE BY RANDOM CUT AND CROSSFILL", tspga=V1_ProcreateByCrossoverWithRandomPoints())


BASELINE:                                     
 Average result fitness: 25.966.
 Average execution time: 0.036s.

PROCREATE BY SIMPLE CROSSOVER:                
 Average result fitness: 26.144.
 Average execution time: 0.035s.

PROCREATE BY RANDOM CUT AND CROSSFILL:        
 Average result fitness: 28.659.
 Average execution time: 0.036s.


**Execution time**: Roughly the same as baseline, simple crossover brings small performance increase.  
**Result fitness**: Using random cut and crossfill seems to decrease the fitness, simple crossover seems to make the resulting fitness more volatile.

**Conclusion**: With the results in mind, I decided to keep the baseline mechanism.

### IGNORE: Changing mutation mechanism

I decided to experiment with two other mechanisms:
1. Incrementing a random element (this results in illegal solutions, but who knows, maybe it will do some good)
2. Swapping slices of random length (generalizes the swap mutation further)

In [23]:
from random import sample


def mutate_by_incrementing(route: Route, city_count: int) -> Route:
    new_route = route.copy()

    pos = randint(0, len(new_route)-1)
    new_route[pos] = new_route[pos] % (city_count-1)

    return new_route

class V1_MutateByIncrementing(V1):
    def mutate(self, route: Route) -> Route:
        return mutate_by_incrementing(route=route, city_count=self.city_count)

In [24]:
from random import randint


def mutate_by_swapping_random_slices(route: Route) -> Route:
    new_route = route.copy()

    slice_length = randint(1, len(route) // 2)

    start1 = randint(0, len(route) - 2 * slice_length)
    end1 = start1 + slice_length
    start2 = end1 + randint(0, len(route) - end1)
    end2 = start2 + slice_length
    
    slice1, slice2 = new_route[start1:end1], new_route[start2:end2]
    new_route[start1:end1], new_route[start2:end2] = slice2, slice1

    return new_route

class V1_MutateBySwappingRandomSlices(V1):
    def mutate(self, route: Route) -> Route:
        return mutate_by_incrementing(route=route, city_count=self.city_count)

In [25]:
measure_baseline()
measure("MUTATE BY INCREMENTING", tspga=V1_MutateByIncrementing())
measure("MUTATE BY SWAPPING RANDOM SLICES", tspga=V1_MutateBySwappingRandomSlices())


BASELINE:                                     
 Average result fitness: 26.276.
 Average execution time: 0.036s.

MUTATE BY INCREMENTING:                       
 Average result fitness: 30.576.
 Average execution time: 0.035s.

MUTATE BY SWAPPING RANDOM SLICES:             
 Average result fitness: 31.057.
 Average execution time: 0.035s.


**Execution time**: Roughly the same as baseline.  
**Result fitness**: Both mutation variants seem to generate worse solutions than the baseline.

**Conclusion**: I will stick with the baseline mutation method.

## (4) Final design and Conclusion

Combining the experimental results the FinalTSPGA will hopefully deliver better results than the other algorithms. 
* The mechanisms will remain unchanged, as none of the others brought any real improvement.
* Population size of 25, parent count of 15, and a mutation rate of 25% will be used.

**Note**: The algorithm now uses all 200 cities instead of the only 50.

In [26]:
class FinalTSPGA(V1):
    def __init__(self, 
                 population_size: int = POPULATION_SIZE, 
                 parent_count: int = PARENT_COUNT, 
                 mutation_rate: float = MUTATION_RATE, 
                 city_count: int = 200):
        super().__init__(population_size, parent_count, mutation_rate, city_count)

In [27]:
measure("BASELINE", tspga=V1(city_count=200), max_generations=400)
measure("FINAL TSPGA", tspga=FinalTSPGA(city_count=200))


BASELINE:                                     
 Average result fitness: 85.300.
 Average execution time: 0.546s.

FINAL TSPGA:                                  
 Average result fitness: 81.497.
 Average execution time: 0.554s.


**Note**: Trying out the final algorithm showed, it took about 4x as long to execute as the baseline, so it seemed only fair to let the baseline execute 4x as many generations. There is still a noticeable improvement with the final algorithm.

### Concluding

After experimenting with different parameters, it's safe to say some changes improved the algorithm and some didn't. 
As the final algorithm brought better results, it's safe to assume that at least some of the observations were valid.

But the experimental results cannot be generalized too well, and have to be treated with caution:
* Averaging only 10 runs still allows for significant variation.
* Furthermore, no measurement of the volatility/variance of the results was taken.
* Not optimizing these parameters automatically, always leaves room for human error.
* Some parameters influence each other, which makes the baseline influence the experimental results (not considered here).